In [2]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers.legacy import SGD

In [3]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('content_2ndgen.json').read())

In [4]:
import json
lemmatizer = WordNetLemmatizer()

In [5]:
words = []
classes = []
documents = []
ignore_letters = ['?','!','.',',']

In [6]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [7]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))


In [8]:
pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))

In [9]:
training = []
output_empty = [0]*len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])


In [10]:
random.shuffle(training)
training = np.array(training, dtype=object)

In [11]:
train_x = list(training[:,0])
train_y = list(training[:,1])

In [12]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu',))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [13]:
sgd = SGD(lr=0.01, decay=1e-6, momentum = 0.9,nesterov=True)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

c:\Users\ashen\anaconda3\envs\detection\lib\site-packages\keras\src\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [14]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs =200, batch_size =5, verbose =1)

Epoch 1/200
105/105 [==============================] - 0s 1ms/step - loss: 4.6944 - accuracy: 0.0630
Epoch 2/200
105/105 [==============================] - 0s 1ms/step - loss: 4.2762 - accuracy: 0.1126
Epoch 3/200
105/105 [==============================] - 0s 1ms/step - loss: 3.9455 - accuracy: 0.1279
Epoch 4/200
105/105 [==============================] - 0s 1ms/step - loss: 3.7063 - accuracy: 0.1775
Epoch 5/200
105/105 [==============================] - 0s 1000us/step - loss: 3.4430 - accuracy: 0.2519
Epoch 6/200
105/105 [==============================] - 0s 980us/step - loss: 3.3189 - accuracy: 0.2672
Epoch 7/200
105/105 [==============================] - 0s 989us/step - loss: 3.1103 - accuracy: 0.3034
Epoch 8/200
105/105 [==============================] - 0s 981us/step - loss: 2.9778 - accuracy: 0.3168
Epoch 9/200
105/105 [==============================] - 0s 985us/step - loss: 2.8470 - accuracy: 0.3340
Epoch 10/200
105/105 [==============================] - 0s 1ms/step - loss: 2.71

In [15]:
model.save('chatbot_model.h5', hist)

c:\Users\ashen\anaconda3\envs\detection\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [1]:
import json
import random
import pickle
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from tensorflow.keras.models import load_model

c:\Users\ashen\anaconda3\envs\detection\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
model = load_model('chatbot_model.h5')

In [3]:
import json
import random
import pickle
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from tensorflow.keras.models import load_model

classes1 = [
    "Null",
    "ADHD",
    "OCD",
]
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
model = load_model('chatbot_model.h5')



def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word ==w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse = True)
    return_list = []
    for r in results:
        return_list.append({'intent':classes[r[0]], 'probability':str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

def game_trig(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse = True)
    return_list1 = []
    for r in results:
        return_list1.append(classes[r[0]])
    
    if return_list1[0]=="greeting":
        return classes1[0]
    else:
        return classes1[1]
print("Go! Bot is running!")


Go! Bot is running!


In [4]:
intents = json.loads(open('content.json').read())
while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    ints1 = game_trig(message)
    print(res)
    print(ints1)

KeyboardInterrupt: Interrupted by user